## Regression Model for USDA Beef Grading

This model will use pytoch and resnet to make a CNN regression model (convolutional neural network)
hay que mantener 5 pasos
1. cargando imagenes y procesarlos
2. modificar el modelo resnet para la regresion
3. entrenar el model
4. evaluar modelo (root mean square)

In [46]:
import torch
import torchvision
import os
import torch.nn as nn
import numpy as np
from torchvision import datasets, transforms, models
import torch.optim as optim
import pandas as pd
from torchvision.models import ResNet18_Weights, VGG16_Weights, AlexNet_Weights
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR

from PIL import Image


from sklearn.metrics import mean_squared_error

print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.4


In [47]:
root_folder =  "Data/Cleaned JPEG Images"
training_folders = [
    "Cargill Dodge 07 20 22 Plant", 
    "Cargill Fort Morgan 03 28 22 Plant",
    "Cargill Schuyler 04 01 22 Plant",
    "Cargill Fort Morgan 07 15 22 Plant"
]

testing_folders= [
    "Cargill Friona 07 18 22 Plant",
    "Cargill Schuyler 07 18 22 Plant",
    "Cargill Fort Morgan 05 12 22 Plant",
]
#df = pd.read_excel(excel_path)

#df.columns = df.columns.str.strip()
#ID_to_score = dict(zip(df['CarcassId'], df['Average']))
#ID_to_score = {f"{carcass_id}.jpg": avg for carcass_id, avg in zip(df['ID'], df['AVG'])}

#print(ID_to_score.keys())

def DataPrepare(root_folder, input_folders):
    all_image_addresses= []
    all_Scores = []
    total_image_number = 0
    for i in range((len(input_folders))):
        img_folder_name = root_folder + "/" +  input_folders[i] + "/Images/"
        Scores_name = root_folder + "/" + input_folders[i] + "/data.xlsx"
        df = pd.read_excel(Scores_name)
        for _, _, files in os.walk(img_folder_name):
            for filename in files:
                total_image_number += 1
                all_image_addresses.append(img_folder_name + filename)
                Score_value = df.at[int(df.index[df['ID'] == filename[0:-4]][0]), 'AVG']
                all_Scores.append(Score_value)
    return all_image_addresses, all_Scores, total_image_number

class ImageDataset(Dataset):
    def __init__(self, root_folder, input_folders, transform =None):
        self.img_dirs, self.scores, self.img_num = DataPrepare(root_folder, input_folders)
        self.transform = transform
                
    def __len__(self):
        return self.img_num
        
    def __getitem__(self, idx):
        image = Image.open(self.img_dirs[idx])
        if self.transform:
            image = self.transform(image)
        score = self.scores[idx] / 1500
        return image, score
      

# Transforms and model

In [48]:
general_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

traindataset = ImageDataset(root_folder, training_folders, transform=general_transform)
traindataloader= DataLoader(traindataset, batch_size=32, shuffle=True)

evaluatedataset= ImageDataset(root_folder, testing_folders, transform=general_transform)
evaluatedataloader= DataLoader(evaluatedataset, batch_size=32, shuffle=False)

In [49]:
   #load pretrained resnet model
resnet18 = models.resnet18(weights=ResNet18_Weights.DEFAULT)
vgg16 = models.vgg16(weights=VGG16_Weights.DEFAULT)
alexnet = models.alexnet(weights=AlexNet_Weights.DEFAULT)

num_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_features,1) #output 1 for regression

num_features_vgg = vgg16.classifier[6].in_features
vgg16.classifier[6] = nn.Linear(num_features_vgg, 1)

num_features_alexnet = alexnet.classifier[6].in_features
alexnet.classifier[6] = nn.Linear(num_features_alexnet, 1)


#LOAD IN THE GPU (MUY IMPORTANTE NO OLVIDES)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

resnet18 = resnet18.to(device)
vgg16 = vgg16.to(device)
alexnet = alexnet.to(device)

cuda


# Training Loop

In [50]:
def train_model(model, num_epochs, dataloader):

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr = 0.001)
    scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

    num_epochs = 65
    model_name = model.__class__.__name__  # Get the model class name
    print(f"Training dataset on model: {model_name}")
    for epoch in range(num_epochs):
        running_loss =0.0
        for images, scores in dataloader:
            images = images.to(device)
            scores = scores.to(device).view(-1,1).float()
            
            optimizer.zero_grad()
            
            outputs = resnet18(images)
            loss = criterion(outputs, scores)
            
            #backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
        scheduler.step()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')
        
    print(f'Training completed! for ({model_name})')
    return model

resnet18 = train_model(resnet18, 65, traindataloader)
vgg16 = train_model(vgg16, 65, traindataloader)
alexnet = train_model(alexnet, 65, traindataloader)

Training dataset on model: ResNet
Epoch [1/65], Loss: 0.9362
Epoch [2/65], Loss: 0.0104
Epoch [3/65], Loss: 0.0042
Epoch [4/65], Loss: 0.0029
Epoch [5/65], Loss: 0.0028
Epoch [6/65], Loss: 0.0024
Epoch [7/65], Loss: 0.0023
Epoch [8/65], Loss: 0.0020
Epoch [9/65], Loss: 0.0019
Epoch [10/65], Loss: 0.0025
Epoch [11/65], Loss: 0.0017
Epoch [12/65], Loss: 0.0019
Epoch [13/65], Loss: 0.0019
Epoch [14/65], Loss: 0.0015
Epoch [15/65], Loss: 0.0014
Epoch [16/65], Loss: 0.0016
Epoch [17/65], Loss: 0.0014
Epoch [18/65], Loss: 0.0013
Epoch [19/65], Loss: 0.0022
Epoch [20/65], Loss: 0.0026
Epoch [21/65], Loss: 0.0009
Epoch [22/65], Loss: 0.0007
Epoch [23/65], Loss: 0.0008
Epoch [24/65], Loss: 0.0009
Epoch [25/65], Loss: 0.0007
Epoch [26/65], Loss: 0.0007
Epoch [27/65], Loss: 0.0007
Epoch [28/65], Loss: 0.0006
Epoch [29/65], Loss: 0.0007
Epoch [30/65], Loss: 0.0006
Epoch [31/65], Loss: 0.0006
Epoch [32/65], Loss: 0.0007
Epoch [33/65], Loss: 0.0007
Epoch [34/65], Loss: 0.0006
Epoch [35/65], Loss: 0.

KeyboardInterrupt: 

# Evaluation

In [11]:
def evaluate_model(model, dataloader):
    model_name = model.__class__.__name__  # Get the model class name
    model.eval()
    all_preds = []
    all_scores = []
    with torch.no_grad():
        for images, scores in dataloader:
            images = images.to(device)
            scores = scores.to(device).view(-1,1).float()
            outputs = model(images)
            all_preds.append(outputs.cpu().numpy())
            all_scores.append(scores.cpu().numpy())
            
    all_preds = np.concatenate(all_preds)
    all_scores = np.concatenate(all_scores)
    mse = mean_squared_error(all_scores, all_preds)
    rmse = np.sqrt(mse)
    
    print(f'RMSE ({model_name}): {rmse*1500:.4f}')
    return rmse

evaluate_model(resnet18, evaluatedataloader)
evaluate_model(vgg16, evaluatedataloader)
evaluate_model(alexnet, evaluatedataloader)

RMSE: 0.0383


0.038317673

# Misc / Junk Code